In [110]:
import requests
import asyncio
import aiohttp
import copy
import pandas as pd
from datetime import datetime
from db_connector import DB
from logger import logging


#apify_client = ApifyClient(token)

# Start an actor and wait for it to finish
#task_call = apify_client.task('yellow_saint~instagram-scraper-task').call(task_input=obj_list[0])

# Fetch results from the actor run's default dataset
#dataset_items = apify_client.dataset(task_call['defaultDatasetId']).list_items().items

In [65]:
D = DB()
list_to_scan = D.pull(db = "Clean",document="Creators",filter= {},colunm={"_id":0,"username":1})
for j in list_to_scan:
    to_scan_dict = {"id":j["username"],"priority":1}
    D.push(to_scan_dict,"Clean","To_scan")

In [119]:
class Scrape():
    Task_URL = "https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/input?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE"
    Scrape_Link_dataset = "https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE"
    Insta_Link = 'https://www.instagram.com/{x}/'



    


    def __init__(self) -> None:
        self.Scanned_count = 0
        self.inputs  =  requests.get(self.Task_URL).json()
        self.DB = DB()
        logging.info("DB connected at Scraper")
        
        
        
    def To_scan(self,update_old_accs:bool) -> list:
        if update_old_accs:
            Unsorted_list = self.DB.pull(db = "Clean",document="To_scan",filter= {"priority":1},colunm={"_id":0},limit= 300)
            if len(Unsorted_list) == 0:
                logging.info("No Accs left to update")
                print("No Accs left to update")
        elif not update_old_accs:
            Unsorted_list = self.DB.pull(db = "Clean",document="To_scan",filter= {"priority":2},colunm={"_id":0},limit= 300)
            if len(Unsorted_list) == 0:
                logging.info("No Accs left to scan")
                print("No Accs left to Scan")
        #DB.remove()
        sorted_data = sorted(Unsorted_list, key=lambda x: x['priority'])
        final_list = []
        for i in sorted_data:
            final_list.append(i["id"])
        #Makking sure the list is unique
        final_list = set(final_list)
        final_list = list(final_list)    
        self.To_scan = final_list
        #remove_filter = {'id': {'$in': final_list }}
        #self.DB.remove(db = "Clean",document="To_scan",filter= remove_filter)    
        logging.info("To_scan listed Created")
        self.input_obj_maker()

    
    async def get(self,session: aiohttp.ClientSession,input_obj:dict,url : str) -> dict:

        print(f"Requesting {url}")
        resp = await session.request('POST', url=url,json=input_obj)

        # Note that this may raise an exception for non-2xx responses
        # You can either handle that here, or pass the exception through
        data = await resp.json()
        print(f"Received data for {url}")
        return data


    async def main(self,input_objs,url):
        # Asynchronous context manager.  Prefer this rather
        # than using a different session for each GET request
        async with aiohttp.ClientSession() as session:
            tasks = []
            for obj in input_objs:
                tasks.append(self.get(session=session, url=url,input_obj=obj))
            # asyncio.gather() will wait on the entire task set to be
            # completed.  If you want to process results greedily as they come in,
            # loop over asyncio.as_completed()
            data = await asyncio.gather(*tasks, return_exceptions=True)
            return data
    def Cleaner(self,data) -> None:
        """
            The Scrape function produces a data in the form of 2-d nested Lists -> list[list] 
            To pull the lists out of the main list we use 2 loops to pull them out
            can be done with recrusion but will try later,
            Also traverse the data to find more related profiles to further scan       
        """
        cleaned_data = []
        for i in data:
            for j in i:
                j["Date"] = datetime.today() 
                cleaned_data.append(j)
        self.Scanned_count = self.Scanned_count + len(cleaned_data)        
        self.DB.push_many(data=cleaned_data,db="Raw_data",document="Creator")

        for i in cleaned_data:

            for j in i["relatedProfiles"]:
                    to_scan_dict = {"id":j["username"],"priority":2}    
                    self.DB.push(to_scan_dict,"Clean","To_scan")
            
      
                
    
    def url_maker(self) -> list:
        print("entered Url Maker")
        url_list = []
        for i in self.To_scan:
            url_list.append(Insta_Link.format(x=i))
        return url_list   
    def input_obj_maker(self) -> None:
        print("entered input obj maker")
        self.obj_list = []
        list_of_usernames = self.url_maker()
        block_size = 10
        blocks = [list_of_usernames[i:i+block_size] for i in range(0, len(list_of_usernames), block_size)]
        for i in blocks:
            temp = copy.deepcopy(self.inputs)
            temp["directUrls"] =  i
            self.obj_list.append(temp)  
    async def scrape(self,no_of_accs:int = 300, Update_old_accs:bool =False):
    
        self.To_scan(Update_old_accs)
        #while self.Scanned_count <= no_of_accs:

            #data = await self.main(self.obj_list,self.Scrape_Link_dataset)
            #self.data = data
            #self.Cleaner(self.data)
        

In [120]:

obj = Scrape()
await obj.scrape(Update_old_accs=False)


No Accs left to Scan
entered input obj maker
entered Url Maker


In [113]:
obj.obj_list

[]

In [68]:
await obj.scrape()

Requesting https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE
Requesting https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE
Requesting https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE
Requesting https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE
Requesting https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE
Requesting https://api.apify.com/v2/actor-tasks/yellow_saint~instagram-scraper-task/run-sync-get-dataset-items?token=apify_api_qIc4Lwctqfr2WAFjmyQZ2mwd1zYhwO0JFLvE
Requesting https

In [58]:
cleaned_data  = []
data = obj.data
for i in data:
            for j in i:
                j["Date"] = datetime.today() 
                cleaned_data.append(j)
        #self.DB.push_many(data=cleaned_data,db="Raw_data",document="Creator")

for i in cleaned_data:

        for j in i["relatedProfiles"]:
            to_scan_dict = {"id":j["username"],"priority":2}
            print(to_scan_dict)

{'id': 'digitalpargat', 'priority': 2}
{'id': 'flywithyogesh', 'priority': 2}
{'id': 'mazhar_10x', 'priority': 2}
{'id': 'themahendra09', 'priority': 2}
{'id': 'theshivansh_10x', 'priority': 2}
{'id': 'digital_aman89', 'priority': 2}
{'id': 'yash_kukreja28', 'priority': 2}
{'id': 'dpksoni._', 'priority': 2}
{'id': 'akdanimesh_', 'priority': 2}
{'id': 'priyanka_parihaarr', 'priority': 2}
{'id': 'moneyprenure_shubh', 'priority': 2}
{'id': 'digitalmanoranjan_', 'priority': 2}
{'id': 'technoprashant1', 'priority': 2}
{'id': 'deepaksafi_official', 'priority': 2}
{'id': 'growthaddicted.official', 'priority': 2}
{'id': 'idigitalpreneur', 'priority': 2}
{'id': 'subropreneur', 'priority': 2}
{'id': 'satyanandarya_', 'priority': 2}
{'id': 'neelammeenaa_', 'priority': 2}
{'id': 'aakankshas0ni', 'priority': 2}
{'id': 'priti_priyedarshni', 'priority': 2}
{'id': 'muskanchawlaaaa', 'priority': 2}
{'id': 'punam_moond', 'priority': 2}
{'id': 'digitalpargat_punjabi', 'priority': 2}
{'id': 'bizgurukul', 

KeyError: 'relatedProfiles'

In [43]:
data[1][0]

{'inputUrl': 'https://www.instagram.com/harshhrane/',
 'id': '2542608356',
 'username': 'harshhrane',
 'url': 'https://www.instagram.com/harshhrane',
 'fullName': 'HARSH RANE🌵',
 'biography': '21 | Actor | Content Creator 🎥\nCreating the Life of my dreams one day at a time.\nManaged by @monkentertainment \nEmail for Collaboration 📩\nYouTube 👇🏻',
 'externalUrl': 'https://www.youtube.com/channel/UCrn3qtZg0k6hM8EN8GPJHHA',
 'externalUrlShimmed': 'https://l.instagram.com/?u=https%3A%2F%2Fwww.youtube.com%2Fchannel%2FUCrn3qtZg0k6hM8EN8GPJHHA&e=AT16mA9ynDUNYV8Ko687PzwBiG_ZfCaNmSY0dZDAdt8iogL914jouZsROfApQvsrJo9WHTRg_p9mmbFLRocrzJ7bXFdqWl3Z',
 'followersCount': 875933,
 'followsCount': 588,
 'hasChannel': False,
 'highlightReelCount': 8,
 'isBusinessAccount': False,
 'joinedRecently': False,
 'businessCategoryName': 'Actor',
 'private': False,
 'verified': False,
 'profilePicUrl': 'https://instagram.fbhd1-2.fna.fbcdn.net/v/t51.2885-19/368099617_1705099199995379_3804098956296091739_n.jpg?stp=ds